# VisibleMale: 3D aligment for volumes 

Import common libraries

In [3]:
import os,sys,pickle,random,threading,time
import platform,subprocess,glob,datetime
import numpy as np
import cv2

In [8]:
# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
%gui qt
%matplotlib qt
print([it for it in sys.modules.keys() if "pyqt" in it.lower()])

['PyQt5', 'PyQt5.sip', 'PyQt5.QtCore', 'PyQt5.QtGui', 'PyQt5.QtWidgets', 'PyQt5.QtSvg']


In [9]:
# must be a separate cell (DO NOT MOVE FROM HERE)
import time
time.sleep(2)

Import OpenVisus and slampy (both open source)

In [12]:
# hardcoded path (CHANGE AS NEEDED)
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
sys.path.append(r"C:\projects\OpenVisus\VisoarAgExplorer")

from OpenVisus                        import *
from OpenVisus.gui                    import *
from OpenVisus.image_utils            import *

from slampy.extract_keypoints         import *
from slampy.gui_utils                 import *
from slampy.image_utils               import *
from slampy.image_provider            import *

Find all images to be stitched, predeclare with and height of the raw images

In [13]:
Width,Height= 2048, 1216
filenames=sorted(glob.glob("D:/GoogleSci/visus_dataset/male/RAW/Fullcolor/fullbody/*.raw"))
Depth=len(filenames)
print("Found",len(filenames),"files")
print("Width",Width,"Height",Height,"Depth",Depth)

Found 1878 files
Width 2048 Height 1216 Depth 1878


Add an utility to load RAW images

In [14]:
def LoadRawImage(filename):
	print("Loading raw image",filename,"...")
	RGB=np.fromfile(filename, dtype=np.uint8)
	Assert(len(RGB)==Width*Height*3) 
	
	# original images are RRR...GGG...BBB
	RGB=RGB.reshape((3, Height, Width)) 
	
	# split channels
	R,G,B=RGB[0,:,:],RGB[1,:,:],RGB[2,:,:]
    
	return [R,G,B]

I have a problem with 12 black VisusbleMale slices. They are all black. Just exclude them.

In [15]:
filenames[1423:1435]=[filenames[1422]]*12

Define density that is the scaling factor for pixels
see https://www.nlm.nih.gov/research/visible/visible_human.html#targetText=These%20images%2C%20each%20approximately%2032,all%201%2C871%20male%20color%20cryosections.&targetText=The%20Visible%20Human%20Female%20data,obtained%20at%200.33%20mm%20intervals.		


In [16]:
Density=[0.33, 0.33, 1.0]

Add a function to remove colored RGB labels at the boundaries (disturbing for the rendering)

In [17]:
def GenerateAlphaRemovingBands(z):
		
	ret=np.zeros((Height,Width),dtype=np.uint8)

	def keepArea(x1,x2,y1,y2):
		ret[int(y1*Height):int(y2*Height),int(x1*Width):int(x2*Width)]=255
			
	
	if   z< 536: keepArea(0.05, 0.95, 0.03, 0.800)
	elif z< 612: keepArea(0.05, 0.95, 0.03, 0.835)
	elif z< 684: keepArea(0.05, 0.95, 0.03, 0.860)
	elif z< 775: keepArea(0.05, 0.95, 0.03, 0.840)
	elif z< 791: keepArea(0.05, 0.95, 0.03, 0.870)
	elif z< 811: keepArea(0.05, 0.95, 0.03, 0.885)
	elif z< 857: keepArea(0.05, 0.95, 0.03, 0.895)
	elif z< 884: keepArea(0.05, 0.95, 0.03, 0.860)
	elif z<1016: keepArea(0.05, 0.95, 0.03, 0.830)
	elif z<1435: keepArea(0.05, 0.95, 0.03, 0.720)
	elif z<1827: keepArea(0.05, 0.82, 0.03, 0.730)
	else:        keepArea(0.05, 0.84, 0.03, 0.800)
	return ret

Create an OpenVisus dataset with unaligned images. Just for quick comparison

In [10]:
bounds=Bounds=BoxNd(PointNd(0,0,0),PointNd(Width*Density[0],Height*Density[1],Depth*Density[2]))

idx_filename="D:/tmp/datasets/male-aligned/visus.idx"

db=CreateIdx(
    url=idx_filename, 
    dim=3,
    dims=(Width,Height,Depth),
    blockperfile=-1,
    filename_template="./visus.bin",
    fields=[Field("data","uint8[4]","row_major")],
    bounds=Position(Bounds))
print("Dataset",filename,"created")

Dataset D:/tmp/datasets/male-aligned/visus.idx created


Example of filling the new dataset with images. 
The OpenVisus library will automatically group them for faster conversion

In [11]:
def generateImages():
	for z in range(Depth):
		R,G,B,=LoadRawImage(filenames[z])
		A=GenerateAlphaRemovingBands(z)
		R[A==0],G[A==0],B[A==0]=0,0,0
		RGBA=InterleaveChannels([R,G,B,A])
		yield RGBA 
db.writeSlabs(generateImages())

Example of using an OpenVisus Viewer inside Jupyter notebook. 

In this case we show the entire dataset interactively.

In [ ]:
viewer=PyViewer()
viewer.open("D:/tmp/datasets/male-aligned/visus.idx")
viewer.setScriptingCode("""
import numpy as np
from OpenVisus.image_utils import SplitChannels
R,G,B,A=SplitChannels(input)
A.fill(255)
A[(R<76) | (G>R) | (B>R)]=0
output=input
output[...,3]=A
""")

# NOTE since the Qt loop is running in the background, there is no need to call run
# viewer.run()

2021-05-28 11:21:24.931156 PyScriptingNode Got in input (0,) float64 origin  
2021-05-28 11:21:24.942158 PyScriptingNode Error
 Traceback (most recent call last):
  File "C:\projects\OpenVisus\build\RelWithDebInfo\OpenVisus\scripting_node.py", line 65, in runJob
    exec(self.code,_globals, _locals)
  File "<string>", line 4, in <module>
ValueError: not enough values to unpack (expected 4, got 0)
 
2021-05-28 11:21:25.113152 PyScriptingNode Got in input (59, 38, 128, 4) uint8 origin Volume 
2021-05-28 11:21:25.117156 PyScriptingNode Output is  (59, 38, 128, 4) uint8 msec 4 
2021-05-28 11:21:25.173466 PyScriptingNode Got in input (59, 76, 128, 4) uint8 origin Volume 
2021-05-28 11:21:25.178477 PyScriptingNode Output is  (59, 76, 128, 4) uint8 msec 5 


Example of running an  viewer first extracting the data, and then rendering the data using a standard VR. 

In [14]:
def ExtractDataAndRemoveBluish(region,quality=-6):
    region=regions['flag_tatoo']
    db=LoadDataset("D:/tmp/datasets/male-aligned/visus.idx")
    logic_box=db.getLogicBox(x=region[0:2],y=region[2:4],z=region[4:6])
    data=db.read(logic_box,quality=quality)	
    bounds=db.getBounds(logic_box)

    # remove the bluish blob around the body
    R,G,B,A=SplitChannels(data)
    A[R<76]  =0
    A[R>76]=255
    A[G>R]=0
    A[B>R]=0
    
    return data,bounds
    
regions={
    'flag_tatoo' : (0.04, 0.95, 0.05,0.73, 0.15,0.15+0.1)	,
    'head' : (0.35,0.65, 0.18,0.82, 0.0,0.2)	,
    'left_arm' : (0.0, 0.4, 0.1,0.8, 0.0,0.3),
    'full' : (0.00,1.00, 0.00,1.00, 0.00,1.00)
}

region=regions['flag_tatoo']
data,bounds=ExtractDataAndRemoveBluish(region)
viewer=PyViewer()
viewer.addGLCamera("glcamera", viewer.getRoot(),"lookat")
viewer.addVolumeRender(data, bounds)
# viewer.run()

Adding Volume render...
done in  4 msec


Now let's show how to use a 3d aligment algorith.
Let's define some internal configurations we will use later.

In [17]:
config={
    'energy_size': 1280,
    'min_num_keypoints':3000,
    'max_num_keypoints':6000,
    'anms':1300,
    'max_reproj_error':0.01, 
    'ratio_check':0.80,
    'ba_tolerance':0.005,
    'fov':60.0
}

Create the slam

In [19]:
slam=Slam()
slam.width   = Width
slam.height  = Height
slam.dtype   = DType.fromString("uint8[3]")

Guessing calibration. Note that for 3d the calibration is not used at all, at least for now.

In [20]:
f=slam.width * (0.5 / math.tan(0.5* math.radians(config['fov'])))
cx,cy=slam.width*0.5,slam.height*0.5
slam.calibration = Calibration(f,cx,cy)
slam.calibration.bFixed = False 

Decide where to store aligned images

In [22]:
cache_dir="D:/tmp/datasets/male-aligned/aligned"
os.makedirs(cache_dir,exist_ok=True)

Add cameras to the slam

In [21]:
for id,filename in enumerate(filenames):
    camera=Camera()
    camera.id=id
    camera.color = Color.random()
    camera.filenames.append(filename)
    slam.addCamera(camera)

Extract keypoints for each image

In [28]:
extractor=ExtractKeyPoints(
    config['min_num_keypoints'], 
    config['max_num_keypoints'], 
    config['anms'])

# iterate for all cameras...
for camera in slam.cameras:
    
    keypoint_filename = cache_dir+"/keypoints/%04d" % (camera.id,)
    
    # load cached kepoints
    if slam.loadKeyPoints(camera, keypoint_filename):
        print("Using cached keypoints",camera.id)
        continue

    print("Generating keypoints",camera.id)
    R,G,B = LoadRawImage(camera.filenames[0])
    A=GenerateAlphaRemovingBands(camera.id)
    R[A==0],G[A==0],B[A==0]=0,0,0
    RGB=InterleaveChannels([R,G,B])
    
    # energy:= a grayscaled image
    energy=(0.3 * R + 0.59 * G + 0.11 * B).astype(np.uint8)
    
    # detect keypoints on a downsampled energy
    energy=ResizeImage(energy, config['energy_size'])
    (keypoints,descriptors)=extractor.doExtract(energy)
    Assert(keypoints)
    
    # scaling factor from keypoints in the downscaled resolution to fullres
    vs=slam.width/float(energy.shape[1])

    # add keypoints to the camera
    camera.keypoints.clear()
    camera.keypoints.reserve(len(keypoints))
    for keypoint in keypoints:
        camera.keypoints.push_back(KeyPoint(vs*keypoint.pt[0], vs*keypoint.pt[1], keypoint.size, keypoint.angle, keypoint.response, keypoint.octave, keypoint.class_id))
    camera.descriptors=Array.fromNumPy(descriptors,TargetDim=2) 

    # for future use (i.e. I won't recompute them from scratch)
    slam.saveKeyPoints(camera,keypoint_filename)  

Cached keypoints 0
Cached keypoints 1
Cached keypoints 2
Cached keypoints 3
Cached keypoints 4
Cached keypoints 5
Cached keypoints 6
Cached keypoints 7
Cached keypoints 8
Cached keypoints 9
Cached keypoints 10
Cached keypoints 11
Cached keypoints 12
Cached keypoints 13
Cached keypoints 14
Cached keypoints 15
Cached keypoints 16
Cached keypoints 17
Cached keypoints 18
Cached keypoints 19
Cached keypoints 20
Cached keypoints 21
Cached keypoints 22
Cached keypoints 23
Cached keypoints 24
Cached keypoints 25
Cached keypoints 26
Cached keypoints 27
Cached keypoints 28
Cached keypoints 29
Cached keypoints 30
Cached keypoints 31
Cached keypoints 32
Cached keypoints 33
Cached keypoints 34
Cached keypoints 35
Cached keypoints 36
Cached keypoints 37
Cached keypoints 38
Cached keypoints 39
Cached keypoints 40
Cached keypoints 41
Cached keypoints 42
Cached keypoints 43
Cached keypoints 44
Cached keypoints 45
Cached keypoints 46
Cached keypoints 47
Cached keypoints 48
Cached keypoints 49
Cached key

Cached keypoints 398
Cached keypoints 399
Cached keypoints 400
Cached keypoints 401
Cached keypoints 402
Cached keypoints 403
Cached keypoints 404
Cached keypoints 405
Cached keypoints 406
Cached keypoints 407
Cached keypoints 408
Cached keypoints 409
Cached keypoints 410
Cached keypoints 411
Cached keypoints 412
Cached keypoints 413
Cached keypoints 414
Cached keypoints 415
Cached keypoints 416
Cached keypoints 417
Cached keypoints 418
Cached keypoints 419
Cached keypoints 420
Cached keypoints 421
Cached keypoints 422
Cached keypoints 423
Cached keypoints 424
Cached keypoints 425
Cached keypoints 426
Cached keypoints 427
Cached keypoints 428
Cached keypoints 429
Cached keypoints 430
Cached keypoints 431
Cached keypoints 432
Cached keypoints 433
Cached keypoints 434
Cached keypoints 435
Cached keypoints 436
Cached keypoints 437
Cached keypoints 438
Cached keypoints 439
Cached keypoints 440
Cached keypoints 441
Cached keypoints 442
Cached keypoints 443
Cached keypoints 444
Cached keypoi

Cached keypoints 790
Cached keypoints 791
Cached keypoints 792
Cached keypoints 793
Cached keypoints 794
Cached keypoints 795
Cached keypoints 796
Cached keypoints 797
Cached keypoints 798
Cached keypoints 799
Cached keypoints 800
Cached keypoints 801
Cached keypoints 802
Cached keypoints 803
Cached keypoints 804
Cached keypoints 805
Cached keypoints 806
Cached keypoints 807
Cached keypoints 808
Cached keypoints 809
Cached keypoints 810
Cached keypoints 811
Cached keypoints 812
Cached keypoints 813
Cached keypoints 814
Cached keypoints 815
Cached keypoints 816
Cached keypoints 817
Cached keypoints 818
Cached keypoints 819
Cached keypoints 820
Cached keypoints 821
Cached keypoints 822
Cached keypoints 823
Cached keypoints 824
Cached keypoints 825
Cached keypoints 826
Cached keypoints 827
Cached keypoints 828
Cached keypoints 829
Cached keypoints 830
Cached keypoints 831
Cached keypoints 832
Cached keypoints 833
Cached keypoints 834
Cached keypoints 835
Cached keypoints 836
Cached keypoi

Cached keypoints 1180
Cached keypoints 1181
Cached keypoints 1182
Cached keypoints 1183
Cached keypoints 1184
Cached keypoints 1185
Cached keypoints 1186
Cached keypoints 1187
Cached keypoints 1188
Cached keypoints 1189
Cached keypoints 1190
Cached keypoints 1191
Cached keypoints 1192
Cached keypoints 1193
Cached keypoints 1194
Cached keypoints 1195
Cached keypoints 1196
Cached keypoints 1197
Cached keypoints 1198
Cached keypoints 1199
Cached keypoints 1200
Cached keypoints 1201
Cached keypoints 1202
Cached keypoints 1203
Cached keypoints 1204
Cached keypoints 1205
Cached keypoints 1206
Cached keypoints 1207
Cached keypoints 1208
Cached keypoints 1209
Cached keypoints 1210
Cached keypoints 1211
Cached keypoints 1212
Cached keypoints 1213
Cached keypoints 1214
Cached keypoints 1215
Cached keypoints 1216
Cached keypoints 1217
Cached keypoints 1218
Cached keypoints 1219
Cached keypoints 1220
Cached keypoints 1221
Cached keypoints 1222
Cached keypoints 1223
Cached keypoints 1224
Cached key

Cached keypoints 1557
Cached keypoints 1558
Cached keypoints 1559
Cached keypoints 1560
Cached keypoints 1561
Cached keypoints 1562
Cached keypoints 1563
Cached keypoints 1564
Cached keypoints 1565
Cached keypoints 1566
Cached keypoints 1567
Cached keypoints 1568
Cached keypoints 1569
Cached keypoints 1570
Cached keypoints 1571
Cached keypoints 1572
Cached keypoints 1573
Cached keypoints 1574
Cached keypoints 1575
Cached keypoints 1576
Cached keypoints 1577
Cached keypoints 1578
Cached keypoints 1579
Cached keypoints 1580
Cached keypoints 1581
Cached keypoints 1582
Cached keypoints 1583
Cached keypoints 1584
Cached keypoints 1585
Cached keypoints 1586
Cached keypoints 1587
Cached keypoints 1588
Cached keypoints 1589
Cached keypoints 1590
Cached keypoints 1591
Cached keypoints 1592
Cached keypoints 1593
Cached keypoints 1594
Cached keypoints 1595
Cached keypoints 1596
Cached keypoints 1597
Cached keypoints 1598
Cached keypoints 1599
Cached keypoints 1600
Cached keypoints 1601
Cached key

Add local images. A local camera is a nearby camera (an edge in the slam graph).
In 3d two consecutive images are local each other.

In [29]:
for I in range(len(slam.cameras)-1):
    camera1=slam.cameras[I+0]
    camera2=slam.cameras[I+1]
    camera1.addLocalCamera(camera2)

Find  matches between local cameras

In [30]:
for camera2 in slam.cameras:
    for camera1 in camera2.getAllLocalCameras():
        
        if camera1.id >= camera2.id:
            continue
            
        matches, H21, error_message=FindMatches(
            slam.width,slam.height,
            camera1.id,[(k.x, k.y) for k in camera1.keypoints],Array.toNumPy(camera1.descriptors), 
            camera2.id,[(k.x, k.y) for k in camera2.keypoints],Array.toNumPy(camera2.descriptors),
            config['max_reproj_error'] * slam.width, 
            config['ratio_check'])

        if error_message:
            print("Error in finding matches",camera1.id,camera2.id,len(matches), error_message)
            camera2.getEdge(camera1).setMatches([],error_message)
            raise Exception("should not happen")
        else:
            print("OK matches",camera1.id,camera2.id,len(matches), "OK")
            matches=[Match(match.queryIdx,match.trainIdx, match.imgIdx, match.distance) for match in matches]
            camera2.getEdge(camera1).setMatches(matches,str(len(matches)))

0/1 knnMatch(1301,1301) matches1(134) matches2(120) SymmetricCheck(72) nmatches(57) in 15 msec
OK matches 0 1 57 OK
1/2 knnMatch(1301,1301) matches1(194) matches2(189) SymmetricCheck(119) nmatches(95) in 13 msec
OK matches 1 2 95 OK
2/3 knnMatch(1301,1301) matches1(217) matches2(214) SymmetricCheck(154) nmatches(132) in 35 msec
OK matches 2 3 132 OK
3/4 knnMatch(1301,1301) matches1(208) matches2(229) SymmetricCheck(153) nmatches(130) in 14 msec
OK matches 3 4 130 OK
4/5 knnMatch(1301,1301) matches1(245) matches2(252) SymmetricCheck(179) nmatches(167) in 15 msec
OK matches 4 5 167 OK
5/6 knnMatch(1301,1301) matches1(351) matches2(355) SymmetricCheck(280) nmatches(269) in 18 msec
OK matches 5 6 269 OK
6/7 knnMatch(1301,1301) matches1(414) matches2(395) SymmetricCheck(320) nmatches(300) in 21 msec
OK matches 6 7 300 OK
7/8 knnMatch(1301,1301) matches1(396) matches2(402) SymmetricCheck(320) nmatches(305) in 19 msec
OK matches 7 8 305 OK
8/9 knnMatch(1301,1301) matches1(413) matches2(423) S

OK matches 71 72 394 OK
72/73 knnMatch(1301,1301) matches1(513) matches2(491) SymmetricCheck(433) nmatches(413) in 26 msec
OK matches 72 73 413 OK
73/74 knnMatch(1301,1301) matches1(505) matches2(499) SymmetricCheck(436) nmatches(384) in 25 msec
OK matches 73 74 384 OK
74/75 knnMatch(1301,1301) matches1(485) matches2(477) SymmetricCheck(413) nmatches(394) in 24 msec
OK matches 74 75 394 OK
75/76 knnMatch(1301,1301) matches1(476) matches2(488) SymmetricCheck(413) nmatches(398) in 25 msec
OK matches 75 76 398 OK
76/77 knnMatch(1301,1301) matches1(483) matches2(488) SymmetricCheck(422) nmatches(390) in 23 msec
OK matches 76 77 390 OK
77/78 knnMatch(1301,1301) matches1(477) matches2(494) SymmetricCheck(417) nmatches(407) in 24 msec
OK matches 77 78 407 OK
78/79 knnMatch(1301,1301) matches1(441) matches2(458) SymmetricCheck(381) nmatches(367) in 23 msec
OK matches 78 79 367 OK
79/80 knnMatch(1301,1301) matches1(448) matches2(439) SymmetricCheck(366) nmatches(286) in 22 msec
OK matches 79 80

OK matches 142 143 316 OK
143/144 knnMatch(1301,1301) matches1(371) matches2(362) SymmetricCheck(306) nmatches(295) in 19 msec
OK matches 143 144 295 OK
144/145 knnMatch(1301,1301) matches1(390) matches2(379) SymmetricCheck(325) nmatches(316) in 20 msec
OK matches 144 145 316 OK
145/146 knnMatch(1301,1301) matches1(384) matches2(368) SymmetricCheck(305) nmatches(295) in 19 msec
OK matches 145 146 295 OK
146/147 knnMatch(1301,1301) matches1(355) matches2(365) SymmetricCheck(302) nmatches(277) in 18 msec
OK matches 146 147 277 OK
147/148 knnMatch(1301,1301) matches1(382) matches2(360) SymmetricCheck(312) nmatches(299) in 20 msec
OK matches 147 148 299 OK
148/149 knnMatch(1301,1301) matches1(354) matches2(347) SymmetricCheck(298) nmatches(274) in 20 msec
OK matches 148 149 274 OK
149/150 knnMatch(1301,1301) matches1(336) matches2(353) SymmetricCheck(284) nmatches(244) in 18 msec
OK matches 149 150 244 OK
150/151 knnMatch(1301,1301) matches1(377) matches2(357) SymmetricCheck(290) nmatches(

OK matches 211 212 487 OK
212/213 knnMatch(1301,1301) matches1(645) matches2(633) SymmetricCheck(563) nmatches(557) in 35 msec
OK matches 212 213 557 OK
213/214 knnMatch(1301,1301) matches1(640) matches2(616) SymmetricCheck(563) nmatches(485) in 33 msec
OK matches 213 214 485 OK
214/215 knnMatch(1301,1301) matches1(594) matches2(597) SymmetricCheck(532) nmatches(505) in 29 msec
OK matches 214 215 505 OK
215/216 knnMatch(1301,1301) matches1(607) matches2(618) SymmetricCheck(539) nmatches(527) in 31 msec
OK matches 215 216 527 OK
216/217 knnMatch(1301,1301) matches1(621) matches2(619) SymmetricCheck(532) nmatches(525) in 31 msec
OK matches 216 217 525 OK
217/218 knnMatch(1301,1301) matches1(596) matches2(591) SymmetricCheck(511) nmatches(505) in 29 msec
OK matches 217 218 505 OK
218/219 knnMatch(1301,1301) matches1(558) matches2(568) SymmetricCheck(483) nmatches(472) in 28 msec
OK matches 218 219 472 OK
219/220 knnMatch(1301,1301) matches1(560) matches2(531) SymmetricCheck(469) nmatches(

OK matches 277 278 475 OK
278/279 knnMatch(1301,1301) matches1(569) matches2(581) SymmetricCheck(489) nmatches(402) in 30 msec
OK matches 278 279 402 OK
279/280 knnMatch(1301,1301) matches1(578) matches2(562) SymmetricCheck(492) nmatches(425) in 28 msec
OK matches 279 280 425 OK
280/281 knnMatch(1301,1301) matches1(596) matches2(605) SymmetricCheck(519) nmatches(484) in 34 msec
OK matches 280 281 484 OK
281/282 knnMatch(1301,1301) matches1(592) matches2(594) SymmetricCheck(507) nmatches(492) in 32 msec
OK matches 281 282 492 OK
282/283 knnMatch(1301,1301) matches1(545) matches2(541) SymmetricCheck(471) nmatches(435) in 27 msec
OK matches 282 283 435 OK
283/284 knnMatch(1301,1301) matches1(585) matches2(591) SymmetricCheck(515) nmatches(458) in 30 msec
OK matches 283 284 458 OK
284/285 knnMatch(1301,1301) matches1(592) matches2(593) SymmetricCheck(505) nmatches(487) in 51 msec
OK matches 284 285 487 OK
285/286 knnMatch(1301,1301) matches1(587) matches2(592) SymmetricCheck(513) nmatches(

...

OK matches 1588 1589 541 OK
1589/1590 knnMatch(1301,1301) matches1(612) matches2(616) SymmetricCheck(521) nmatches(513) in 35 msec
OK matches 1589 1590 513 OK
1590/1591 knnMatch(1301,1301) matches1(689) matches2(676) SymmetricCheck(608) nmatches(566) in 39 msec
OK matches 1590 1591 566 OK
1591/1592 knnMatch(1301,1301) matches1(642) matches2(649) SymmetricCheck(577) nmatches(573) in 34 msec
OK matches 1591 1592 573 OK
1592/1593 knnMatch(1301,1301) matches1(648) matches2(667) SymmetricCheck(577) nmatches(561) in 37 msec
OK matches 1592 1593 561 OK
1593/1594 knnMatch(1301,1301) matches1(632) matches2(634) SymmetricCheck(556) nmatches(544) in 40 msec
OK matches 1593 1594 544 OK
1594/1595 knnMatch(1301,1301) matches1(665) matches2(656) SymmetricCheck(587) nmatches(572) in 37 msec
OK matches 1594 1595 572 OK
1595/1596 knnMatch(1301,1301) matches1(694) matches2(684) SymmetricCheck(604) nmatches(597) in 40 msec
OK matches 1595 1596 597 OK
1596/1597 knnMatch(1301,1301) matches1(683) matches2(67

OK matches 1653 1654 493 OK
1654/1655 knnMatch(1301,1301) matches1(592) matches2(608) SymmetricCheck(531) nmatches(488) in 33 msec
OK matches 1654 1655 488 OK
1655/1656 knnMatch(1301,1301) matches1(565) matches2(565) SymmetricCheck(487) nmatches(470) in 31 msec
OK matches 1655 1656 470 OK
1656/1657 knnMatch(1301,1301) matches1(511) matches2(512) SymmetricCheck(445) nmatches(436) in 26 msec
OK matches 1656 1657 436 OK
1657/1658 knnMatch(1301,1301) matches1(558) matches2(547) SymmetricCheck(472) nmatches(448) in 29 msec
OK matches 1657 1658 448 OK
1658/1659 knnMatch(1301,1301) matches1(524) matches2(532) SymmetricCheck(452) nmatches(441) in 27 msec
OK matches 1658 1659 441 OK
1659/1660 knnMatch(1301,1301) matches1(471) matches2(466) SymmetricCheck(388) nmatches(361) in 24 msec
OK matches 1659 1660 361 OK
1660/1661 knnMatch(1301,1301) matches1(501) matches2(509) SymmetricCheck(432) nmatches(423) in 25 msec
OK matches 1660 1661 423 OK
1661/1662 knnMatch(1301,1301) matches1(515) matches2(47

OK matches 1717 1718 390 OK
1718/1719 knnMatch(1301,1301) matches1(501) matches2(499) SymmetricCheck(421) nmatches(410) in 27 msec
OK matches 1718 1719 410 OK
1719/1720 knnMatch(1301,1301) matches1(493) matches2(481) SymmetricCheck(417) nmatches(402) in 26 msec
OK matches 1719 1720 402 OK
1720/1721 knnMatch(1301,1301) matches1(498) matches2(503) SymmetricCheck(409) nmatches(399) in 26 msec
OK matches 1720 1721 399 OK
1721/1722 knnMatch(1301,1301) matches1(471) matches2(476) SymmetricCheck(393) nmatches(371) in 24 msec
OK matches 1721 1722 371 OK
1722/1723 knnMatch(1301,1301) matches1(454) matches2(463) SymmetricCheck(370) nmatches(359) in 24 msec
OK matches 1722 1723 359 OK
1723/1724 knnMatch(1301,1301) matches1(441) matches2(438) SymmetricCheck(361) nmatches(334) in 22 msec
OK matches 1723 1724 334 OK
1724/1725 knnMatch(1301,1301) matches1(471) matches2(474) SymmetricCheck(384) nmatches(370) in 24 msec
OK matches 1724 1725 370 OK
1725/1726 knnMatch(1301,1301) matches1(496) matches2(51

OK matches 1787 1788 273 OK
1788/1789 knnMatch(1301,1301) matches1(368) matches2(380) SymmetricCheck(309) nmatches(299) in 20 msec
OK matches 1788 1789 299 OK
1789/1790 knnMatch(1301,1301) matches1(420) matches2(433) SymmetricCheck(338) nmatches(321) in 24 msec
OK matches 1789 1790 321 OK
1790/1791 knnMatch(1301,1301) matches1(432) matches2(431) SymmetricCheck(347) nmatches(331) in 23 msec
OK matches 1790 1791 331 OK
1791/1792 knnMatch(1301,1301) matches1(479) matches2(490) SymmetricCheck(390) nmatches(360) in 25 msec
OK matches 1791 1792 360 OK
1792/1793 knnMatch(1301,1301) matches1(583) matches2(583) SymmetricCheck(486) nmatches(470) in 31 msec
OK matches 1792 1793 470 OK
1793/1794 knnMatch(1301,1301) matches1(547) matches2(563) SymmetricCheck(457) nmatches(417) in 33 msec
OK matches 1793 1794 417 OK
1794/1795 knnMatch(1301,1301) matches1(610) matches2(581) SymmetricCheck(499) nmatches(423) in 33 msec
OK matches 1794 1795 423 OK
1795/1796 knnMatch(1301,1301) matches1(579) matches2(58

OK matches 1853 1854 532 OK
1854/1855 knnMatch(1301,1301) matches1(632) matches2(634) SymmetricCheck(543) nmatches(521) in 35 msec
OK matches 1854 1855 521 OK
1855/1856 knnMatch(1301,1301) matches1(685) matches2(645) SymmetricCheck(565) nmatches(554) in 37 msec
OK matches 1855 1856 554 OK
1856/1857 knnMatch(1301,1301) matches1(692) matches2(682) SymmetricCheck(596) nmatches(583) in 38 msec
OK matches 1856 1857 583 OK
1857/1858 knnMatch(1301,1301) matches1(614) matches2(608) SymmetricCheck(523) nmatches(511) in 33 msec
OK matches 1857 1858 511 OK
1858/1859 knnMatch(1301,1301) matches1(575) matches2(573) SymmetricCheck(489) nmatches(482) in 30 msec
OK matches 1858 1859 482 OK
1859/1860 knnMatch(1301,1301) matches1(567) matches2(583) SymmetricCheck(485) nmatches(472) in 30 msec
OK matches 1859 1860 472 OK
1860/1861 knnMatch(1301,1301) matches1(626) matches2(638) SymmetricCheck(547) nmatches(534) in 34 msec
OK matches 1860 1861 534 OK
1861/1862 knnMatch(1301,1301) matches1(643) matches2(63

I should not have any disconnected camera in 3d so the following is commented:

In [42]:
# # slam.removeDisconnectedCameras()

Run bundle adjustment. Run two times, the first one just to remove coarse outliers, the second one to get a better solution

In [32]:
slam.bundleAdjustment(10.0*config['ba_tolerance'],"offset")
slam.removeOutlierMatches(config['max_reproj_error'])

slam.bundleAdjustment(1.0*config['ba_tolerance'],"offset")
slam.removeOutlierMatches(config['max_reproj_error']) 
    
# I do not have disconnected cameras
#slam.removeDisconnectedCameras()

# I don't have distortions in 3d
#slam.removeCamerasWithTooMuchSkew()

Extract the bundle adjustment results i.e. the offset of the images

In [34]:
offsets=[(round(camera.homography(0,2)),round(camera.homography(1,2))) for camera in slam.cameras ]
print(offsets)

[(-7, -6), (-8, -6), (-8, -6), (-8, -6), (-8, -6), (-8, -6), (-8, -6), (-8, -6), (-7, -5), (-7, -6), (-7, -5), (-7, -5), (-6, -6), (-6, -5), (-6, -6), (-6, -6), (-6, -6), (-6, -7), (-6, -6), (-6, -6), (-6, -6), (-6, -6), (-6, -5), (-6, -5), (-5, -6), (-6, -6), (-5, -6), (-5, -6), (-5, -6), (-5, -6), (-5, -5), (-5, -5), (-4, -5), (-4, -5), (-4, -5), (-4, -5), (-5, -5), (-4, -5), (-4, -5), (-4, -5), (-4, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -6), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -5), (-5, -4), (-5, -4), (-5, -4), (-5, -4), (-4, -4), (-4, -4), (-4, -4), (-4, -5), (-4, -4), (-4, -4), (-4, -4), (-4, -4), (-4, -4), (-4, -4), (-4, -4), (-4, -4), (-5, -3), (-5, -3), (-5, -3), (-5, -3), (-5, -3), (-5, -2), (-5, -2), (-5, -2), (-5, -1), (-5, -2), (-5, -1), (-4, -1), (-4, -1), (-4, 0), (-4, 0), (-4, 0), (-4, 0), (-4, 0), (-3, 1), (-3, 1), (-3, 1), (-3, 1), (-3, 1), (-3, 1), (-4, 1), (-4, 1), (-4, 1), (-4, 1), (-4,

Create a new OpenVisus dataset ready to be filled with aligned images

In [ ]:
bounds=BoxNd(PointNd(0,0,0),PointNd(Width*Density[0],Height*Density[1],Depth*Density[2]))
db=CreateIdx(
    url="D:/tmp/datasets/male-aligned/visus.idx", 
    dim=3,
    dims=(Width,Height,Depth),
    blockperfile=-1,
    filename_template="./visus.bin",
    fields=[Field("data","uint8[4]","row_major")],
    bounds=Position(bounds))

Fill the dataset with aligned image. An aligned image is the original image moved by a certain offset

In [35]:
def generator(): 
	for I,camera in enumerate(slam.cameras):
		
		# note: there are few slices that are totally black, going to use them as they are
		offset=offsets[I]
		
		R,G,B=LoadRawImage(camera.filenames[0])
		A=GenerateAlphaRemovingBands(camera.id)
        
        # make sure I stick with the predeclared logic bounds of the idx datsset
		R[A==0],G[A==0],B[A==0]=0,0,0
		x1,x2 = (0,Width)
		y1,y2 = (0,Height)
		X1,X2 = (max([x1+offset[0],0]),min([x2+offset[0],Width]))
		Y1,Y2 = (max([y1+offset[1],0]),min([y2+offset[1],Height]))
		x1,x2=(X1-offset[0],X2-offset[0])
		y1,y2=(Y1-offset[1],Y2-offset[1])
        
		RGBA=np.zeros((Height,Width,4),dtype='uint8')
		RGBA[Y1:Y2, X1:X2, 0]=R[y1:y2,x1:x2]
		RGBA[Y1:Y2, X1:X2, 1]=G[y1:y2,x1:x2]
		RGBA[Y1:Y2, X1:X2, 2]=B[y1:y2,x1:x2]
		RGBA[Y1:Y2, X1:X2, 3]=A[y1:y2,x1:x2]

        # save the aligned image
		# SaveImage(cache_dir+"/aligned/%04d.tif" % (camera.id,), RGBA)

		yield RGBA    
    
generator=GenerateAlignedImages()

Show the new dataset in the OpenVisus viewer

In [37]:
viewer=PyViewer()
viewer.open("D:/tmp/datasets/male-aligned/visus.idx")
viewer.setScriptingCode("""
import numpy as np
from OpenVisus.image_utils import SplitChannels
R,G,B,A=SplitChannels(input)
A.fill(255)
A[(R<76) | (G>R) | (B>R)]=0
output=input
output[...,3]=A
""")
# viewer.run()

2021-04-20 14:08:17.542275 PyScriptingNode Got in input (0,) float64 origin  
2021-04-20 14:08:17.544277 PyScriptingNode Error
 Traceback (most recent call last):
  File "C:\projects\OpenVisus\build\RelWithDebInfo\OpenVisus\scripting_node.py", line 65, in runJob
    exec(self.code,_globals, _locals)
  File "<string>", line 4, in <module>
ValueError: not enough values to unpack (expected 4, got 0)
 
2021-04-20 14:08:17.672008 PyScriptingNode Got in input (118, 76, 256, 4) uint8 origin Volume 
2021-04-20 14:08:17.685018 PyScriptingNode Output is  (118, 76, 256, 4) uint8 msec 14 


Show some aligned pieces using OpenVisus volume renderer.

In [38]:
regions={
    'flag_tatoo' : (0.04, 0.95, 0.05,0.73, 0.15,0.15+0.1)	,
    'head' : (0.35,0.65, 0.18,0.82, 0.0,0.2)	,
    'left_arm' : (0.0, 0.4, 0.1,0.8, 0.0,0.3),
    'full' : (0.00,1.00, 0.00,1.00, 0.00,1.00)
}

region=regions['flag_tatoo']
db=LoadDataset("D:/tmp/datasets/male-aligned/visus.idx")
logic_box=db.getLogicBox(x=region[0:2],y=region[2:4],z=region[4:6])
data=db.read(logic_box,quality=-6)	
bounds=db.getBounds(logic_box)

# remove the bluish blob around the body
R,G,B,A=SplitChannels(data)
A[R<76]  =0
A[R>76]=255
A[G>R]=0
A[B>R]=0

viewer=PyViewer()
viewer.addGLCamera("glcamera", viewer.getRoot(),"lookat")
viewer.addVolumeRender(data, bounds)
#viewer.run()

Adding Volume render...
done in  4 msec
